### 第一步，导入 TensorFlow 包。 

如果你按照实战准备一的步骤配置好了 TensorFlow Python 环境，就可以成功地导入 TensorFlow 包。接下来，你要做的就是定义好训练数据的路径 TRAIN_DATA_URL 了，然后根据你自己训练数据的本地路径，替换参考代码中的路径就可以了。

In [1]:
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "file:///opt/bigdata/lab/recommend-system/datas/modelSamples.csv"
samples_file_path = tf.keras.utils.get_file("modelSamples.csv", TRAIN_DATA_URL)
print(samples_file_path)

/home/bigdata/.keras/datasets/modelSamples.csv


### 第二步是载入训练数据

我们利用 TensorFlow 自带的 CSV 数据集的接口载入训练数据。注意这里有两个比较重要的参数，一个是 label_name，它指定了 CSV 数据集中的标签列。另一个是 batch_size，它指定了训练过程中，一次输入几条训练数据进行梯度下降训练。载入训练数据之后，我们把它们分割成了测试集和训练集。

In [3]:
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="?",
        num_epochs=1,
        ignore_errors=True)
    return dataset


# sample dataset size 110830/12(batch_size) = 9235
raw_samples_data = get_dataset(samples_file_path)

In [5]:
test_dataset = raw_samples_data.take(1000)
train_dataset = raw_samples_data.skip(1000)

In [6]:
raw_samples_data.take(1)

<TakeDataset shapes: (OrderedDict([(movieId, (None,)), (userId, (None,)), (rating, (None,)), (timestamp, (None,)), (releaseYear, (None,)), (movieGenre1, (None,)), (movieGenre2, (None,)), (movieGenre3, (None,)), (movieRatingCount, (None,)), (movieAvgRating, (None,)), (movieRatingStddev, (None,)), (userRatedMovie1, (None,)), (userRatedMovie2, (None,)), (userRatedMovie3, (None,)), (userRatedMovie4, (None,)), (userRatedMovie5, (None,)), (userRatingCount, (None,)), (userAvgReleaseYear, (None,)), (userReleaseYearStddev, (None,)), (userAvgRating, (None,)), (userRatingStddev, (None,)), (userGenre1, (None,)), (userGenre2, (None,)), (userGenre3, (None,)), (userGenre4, (None,)), (userGenre5, (None,))]), (None,)), types: (OrderedDict([(movieId, tf.int32), (userId, tf.int32), (rating, tf.float32), (timestamp, tf.int32), (releaseYear, tf.int32), (movieGenre1, tf.string), (movieGenre2, tf.string), (movieGenre3, tf.string), (movieRatingCount, tf.int32), (movieAvgRating, tf.float32), (movieRatingStddev

### 第三步是载入类别型特征。 


我们用到的类别型特征主要有这三类，分别是 genre、userId 和 movieId。在载入 genre 类特征时，我们采用了 tf.feature_column.categorical_column_with_vocabulary_list 方法把字符串型的特征转换成了 One-hot 特征。在这个转换过程中我们需要用到一个词表，你可以看到我在开头就定义好了包含所有 genre 类别的词表 genre_vocab。

在转换 userId 和 movieId 特征时，我们又使用了 tf.feature_column.categorical_column_with_identity 方法把 ID 转换成 One-hot 特征，这个方法不用词表，它会直接把 ID 值对应的那个维度置为 1。比如，我们输入这个方法的 movieId 是 340，总的 movie 数量是 1001，使用这个方法，就会把这个 1001 维的 One-hot movieId 向量的第 340 维置为 1，剩余的维度都为 0。

为了把稀疏的 One-hot 特征转换成稠密的 Embedding 向量，我们还需要在 One-hot 特征外包裹一层 Embedding 层，你可以看到 tf.feature_column.embedding_column(movie_col, 10) 方法完成了这样的操作，它在把 movie one-hot 向量映射到了一个 10 维的 Embedding 层上。

In [13]:
genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller',
               'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']


GENRE_FEATURES = {
    'userGenre1': genre_vocab,
    'userGenre2': genre_vocab,
    'userGenre3': genre_vocab,
    'userGenre4': genre_vocab,
    'userGenre5': genre_vocab,
    'movieGenre1': genre_vocab,
    'movieGenre2': genre_vocab,
    'movieGenre3': genre_vocab
}


categorical_columns = []
for feature, vocab in GENRE_FEATURES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    emb_col = tf.feature_column.embedding_column(cat_col, 10)
    categorical_columns.append(emb_col)


movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
categorical_columns.append(movie_emb_col)


user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
categorical_columns.append(user_emb_col)

## 第四步是数值型特征的处理。 

这一步非常简单，我们直接把特征值输入到 MLP 内，然后把特征逐个声明为 tf.feature_column.numeric_column 就可以了，不需要经过其他的特殊处理。

In [14]:
numerical_columns = [tf.feature_column.numeric_column('releaseYear'),
                   tf.feature_column.numeric_column('movieRatingCount'),
                     tf.feature_column.numeric_column('movieAvgRating'),
                     tf.feature_column.numeric_column('movieRatingStddev'),
                     tf.feature_column.numeric_column('userRatingCount'),
                     tf.feature_column.numeric_column('userAvgRating'),
                     tf.feature_column.numeric_column('userRatingStddev')]


### 第五步是定义模型结构。 

这一步的实现代码也非常简洁，我们直接利用 DenseFeatures 把类别型 Embedding 特征和数值型特征连接在一起形成稠密特征向量，然后依次经过两层 128 维的全连接层，最后通过 sigmoid 输出神经元产生最终预估值。

In [15]:

preprocessing_layer = tf.keras.layers.DenseFeatures(numerical_columns + categorical_columns)


model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])


### 第六步是定义模型训练相关的参数。 

在这一步中，我们需要设置模型的损失函数，梯度反向传播的优化方法，以及模型评估所用的指标。关于损失函数，我们使用的是二分类问题最常用的二分类交叉熵，优化方法使用的是深度学习中很流行的 adam，最后是评估指标，使用了准确度 accuracy 作为模型评估的指标。

In [16]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

### 第七步是模型的训练和评估。 

TensorFlow 模型的训练过程和 Spark MLlib 一样，都是调用 fit 函数，然后使用 evaluate 函数在测试集上进行评估。不过，这里我们要注意一个参数 epochs，它代表了模型训练的轮数，一轮代表着使用所有训练数据训练一遍，epochs=10 代表着训练 10 遍。

In [17]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
8232/8232 [==============================] - 53s 6ms/step - loss: 6.5381 - accuracy: 0.5514
Epoch 2/10
8232/8232 [==============================] - 49s 6ms/step - loss: 0.7116 - accuracy: 0.6428
Epoch 3/10
8232/8232 [==============================] - 49s 6ms/step - loss: 0.5638 - accuracy: 0.7134
Epoch 4/10
8232/8232 [==============================] - 49s 6ms/step - loss: 0.5353 - accuracy: 0.7335
Epoch 5/10
8232/8232 [==============================] - 49s 6ms/step - loss: 0.5118 - accuracy: 0.7491
Epoch 6/10
8232/8232 [==============================] - 49s 6ms/step - loss: 0.5048 - accuracy: 0.7513
Epoch 7/10
8232/8232 [==============================] - 49s 6ms/step - loss: 0.4919 - accuracy: 0.7585
Epoch 8/10
8232/8232 [==============================] - 51s 6ms/step - loss: 0.4847 - accuracy: 0.7640
Epoch 9/10
8232/8232 [==============================] - 50s 6ms/st

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))